In [1]:
# Core imports and functions
import json
from openai import OpenAI
from read_dataset import read_all_documents
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment
from difflib import SequenceMatcher

# Import API key
from apikeys import open_ai_api_key

# Set up OpenAI client
client = OpenAI(api_key=open_ai_api_key)

def extract_metadata_with_llm(document_content: str, filename: str) -> dict:
    """Extract metadata from document content using OpenAI GPT-4.1"""
    max_content_length = 100000
    truncated_content = document_content[:max_content_length]
    
    prompt = f"""
    You are analyzing a Slovak government document. Please extract the following information and return it as a JSON object in English:
    
    Document filename: {filename}
    Document content: {truncated_content}
    
    Please extract and return the following information in JSON format:
    {{
        "project_name": "English translation of the project name",
        "project_name_slovak": "Original Slovak project name",
        "document_type": "Type of document (e.g., feasibility study, evaluation, financial analysis, etc.)",
        "sector": "Sector of the project. The Sector of the project is one of the following: Building, Transport, IT, Defense, Other: xxx (Specify if Other)",
        "summary": "Brief summary of the document content in English"
    }}
    
    Important notes:
    - If the document is in Slovak, translate the project name to English but also keep the original Slovak version
    - Be specific about the document type based on the content
    - Identify the sector/domain the project belongs to
    - Provide a concise but informative summary
    - Return only valid JSON, no additional text
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[
                {"role": "system", "content": "You are an expert document analyst specializing in Slovak government documents. Extract metadata accurately and return only valid JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=1000
        )
        
        content = response.choices[0].message.content.strip()
        
        try:
            metadata = json.loads(content)
            return metadata
        except json.JSONDecodeError:
            start_idx = content.find('{')
            end_idx = content.rfind('}') + 1
            if start_idx != -1 and end_idx != 0:
                json_str = content[start_idx:end_idx]
                metadata = json.loads(json_str)
                return metadata
            else:
                return {"error": "Could not parse JSON from LLM response", "raw_response": content}
                
    except Exception as e:
        return {"error": f"LLM processing failed: {str(e)}"}

def calculate_similarity(str1: str, str2: str) -> float:
    """Calculate similarity between two strings (0-1 scale) with null safety"""
    if not str1 or not str2:
        return 0.0
    return SequenceMatcher(None, str(str1).lower(), str(str2).lower()).ratio()

def find_fuzzy_matches(new_project_name: str, existing_projects: dict, threshold: float = 0.8) -> list:
    """Find existing projects that match the new project name above threshold"""
    if not new_project_name or not existing_projects:
        return []
    
    matches = []
    for canonical_name, project_data in existing_projects.items():
        if not canonical_name:
            continue
        similarity = calculate_similarity(new_project_name, canonical_name)
        if similarity >= threshold:
            matches.append({
                'canonical_name': canonical_name,
                'similarity': similarity,
                'project_data': project_data
            })
    return sorted(matches, key=lambda x: x['similarity'], reverse=True)

def ask_llm_for_project_matching(new_project_name: str, new_summary: str, existing_projects: dict) -> dict:
    """Ask LLM to determine if new project matches any existing projects"""
    if not new_project_name or not existing_projects:
        return {"is_match": False, "matched_project": None, "confidence": "low", "reasoning": "No project name or existing projects"}
    
    existing_projects_list = []
    for canonical_name, project_data in existing_projects.items():
        if not canonical_name:
            continue
        existing_projects_list.append({
            'name': canonical_name,
            'summary': project_data.get('summary', ''),
            'document_count': len(project_data.get('documents', []))
        })
    
    if not existing_projects_list:
        return {"is_match": False, "matched_project": None, "confidence": "low", "reasoning": "No valid existing projects"}
    
    prompt = f"""
    You are analyzing Slovak government projects. I have a new project and need to determine if it matches any existing projects.
    
    NEW PROJECT:
    Name: {new_project_name}
    Summary: {new_summary}
    
    EXISTING PROJECTS:
    {json.dumps(existing_projects_list, indent=2, ensure_ascii=False)}
    
    Please analyze if the new project is the same as any existing project. Consider:
    - Project names (even if slightly different)
    - Project content and scope
    - Government institutions involved
    - Geographic locations
    - Project phases/stages
    
    Return your analysis as JSON:
    {{
        "is_match": true/false,
        "matched_project": "canonical name if match found, null otherwise",
        "confidence": "high/medium/low",
        "reasoning": "brief explanation of your decision"
    }}
    
    Important: Projects can have different names but be the same project (e.g., different phases, updates, evaluations of the same initiative).
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[
                {"role": "system", "content": "You are an expert at analyzing Slovak government projects and identifying when documents belong to the same project despite different names."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=500
        )
        
        content = response.choices[0].message.content.strip()
        
        try:
            result = json.loads(content)
            return result
        except json.JSONDecodeError:
            start_idx = content.find('{')
            end_idx = content.rfind('}') + 1
            if start_idx != -1 and end_idx != 0:
                json_str = content[start_idx:end_idx]
                result = json.loads(json_str)
                return result
            else:
                return {"is_match": False, "matched_project": None, "confidence": "low", "reasoning": "Could not parse LLM response"}
                
    except Exception as e:
        return {"is_match": False, "matched_project": None, "confidence": "low", "reasoning": f"LLM error: {str(e)}"}

def save_to_excel(data: list, filename: str, sheet_name: str = "Data"):
    """Save data to Excel with formatting"""
    df = pd.DataFrame(data)
    
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        workbook = writer.book
        worksheet = writer.sheets[sheet_name]
        
        # Format headers
        header_font = Font(bold=True, color="FFFFFF")
        header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
        header_alignment = Alignment(horizontal="center", vertical="center")
        
        for cell in worksheet[1]:
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
        
        # Auto-adjust column widths
        for column in worksheet.columns:
            max_length = 0
            column_letter = column[0].column_letter
            
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            
            adjusted_width = min(max_length + 2, 50)
            worksheet.column_dimensions[column_letter].width = adjusted_width
        
        # Format summary column to wrap text
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row):
            if len(row) > 5:  # Summary column (0-indexed)
                summary_cell = row[5]
                summary_cell.alignment = Alignment(wrap_text=True, vertical="top")
    
    print(f"✅ Data saved to {filename}")

print("✅ All core functions loaded successfully!")
print("📋 Available functions:")
print("  - extract_metadata_with_llm()")
print("  - calculate_similarity()")
print("  - find_fuzzy_matches()")
print("  - ask_llm_for_project_matching()")
print("  - save_to_excel()")


✅ All core functions loaded successfully!
📋 Available functions:
  - extract_metadata_with_llm()
  - calculate_similarity()
  - find_fuzzy_matches()
  - ask_llm_for_project_matching()
  - save_to_excel()


In [2]:
# REAL-TIME document processing with immediate project consolidation
print("🚀 Starting REAL-TIME document processing with project consolidation...")
print("Using GPT-4.1 for analysis + immediate project matching")
print("Processing documents one by one with live consolidation")
print("=" * 60)

# Read all documents
documents = read_all_documents("Data")
all_metadata = []
canonical_projects = {}  # Running tally of consolidated projects

print(f"Processing {len(documents)} documents with real-time consolidation...")

for i, (filename, content) in enumerate(documents.items(), 1):
    print(f"\n📄 Processing Document {i}/{len(documents)}: {filename}")
    print(f"Content length: {len(content)} characters")
    
    # Extract metadata using LLM
    metadata = extract_metadata_with_llm(content, filename)
    metadata["filename"] = filename
    metadata["content_length"] = len(content)
    
    if 'error' in metadata:
        print(f"  ❌ Error: {metadata['error']}")
        all_metadata.append(metadata)
        continue
    
    project_name = metadata.get('project_name', '')
    summary = metadata.get('summary', '')
    
    print(f"  📋 Extracted Project: {project_name}")
    
    # REAL-TIME CONSOLIDATION: Check if this project matches existing ones
    print(f"  🔍 Checking for project matches...")
    
    # Step 1: Fuzzy matching
    fuzzy_matches = find_fuzzy_matches(project_name, canonical_projects, threshold=0.8)
    
    if fuzzy_matches:
        print(f"    🔍 Found {len(fuzzy_matches)} fuzzy matches:")
        for match in fuzzy_matches:
            print(f"      - {match['canonical_name']} (similarity: {match['similarity']:.2f})")
    
    # Step 2: LLM matching (compare against ALL existing projects)
    llm_result = ask_llm_for_project_matching(project_name, summary, canonical_projects)
    
    print(f"    🤖 LLM Analysis:")
    print(f"      - Match: {llm_result['is_match']}")
    print(f"      - Confidence: {llm_result['confidence']}")
    print(f"      - Reasoning: {llm_result['reasoning']}")
    
    # Step 3: Decision logic
    matched_project = None
    decision_reason = ""
    
    if llm_result['is_match'] and llm_result['matched_project']:
        # LLM found a match
        matched_project = llm_result['matched_project']
        decision_reason = f"LLM match (confidence: {llm_result['confidence']})"
    elif fuzzy_matches and fuzzy_matches[0]['similarity'] >= 0.9:
        # Very high fuzzy match
        matched_project = fuzzy_matches[0]['canonical_name']
        decision_reason = f"High fuzzy match ({fuzzy_matches[0]['similarity']:.2f})"
    else:
        # No clear match - create new project
        matched_project = None
        decision_reason = "No match found - creating new project"
    
    # Step 4: Update canonical projects
    if matched_project:
        # Add to existing project
        canonical_projects[matched_project]['documents'].append({
            'filename': filename,
            'project_name': project_name,
            'project_name_slovak': metadata.get('project_name_slovak', ''),
            'document_type': metadata.get('document_type', ''),
            'sector': metadata.get('sector', ''),
            'summary': summary,
            'content_length': metadata.get('content_length', 0)
        })
        
        # Update canonical name if new name is better (longer, more descriptive)
        if len(project_name) > len(matched_project) and 'update' not in project_name.lower():
            old_name = matched_project
            canonical_projects[project_name] = canonical_projects.pop(old_name)
            canonical_projects[project_name]['canonical_name'] = project_name
            print(f"    📝 Updated canonical name: '{old_name}' → '{project_name}'")
        
        print(f"    ✅ Added to existing project: {matched_project}")
    else:
        # Create new project
        canonical_projects[project_name] = {
            'canonical_name': project_name,
            'project_name_slovak': metadata.get('project_name_slovak', ''),
            'sector': metadata.get('sector', ''),
            'documents': [{
                'filename': filename,
                'project_name': project_name,
                'project_name_slovak': metadata.get('project_name_slovak', ''),
                'document_type': metadata.get('document_type', ''),
                'sector': metadata.get('sector', ''),
                'summary': summary,
                'content_length': metadata.get('content_length', 0)
            }]
        }
        print(f"    🆕 Created new project: {project_name}")
    
    print(f"    📊 Decision: {decision_reason}")
    
    # Add to all_metadata for compatibility
    all_metadata.append(metadata)
    
    # Show current consolidation status
    print(f"    📈 Current Status: {len(canonical_projects)} consolidated projects")
    print("-" * 60)

print(f"\n🎉 Real-time processing complete!")
print(f"Total documents processed: {len(all_metadata)}")
print(f"Total consolidated projects: {len(canonical_projects)}")

# Show summary
successful_extractions = [m for m in all_metadata if 'error' not in m]
failed_extractions = [m for m in all_metadata if 'error' in m]

print(f"\n📊 Summary:")
print(f"✅ Successful extractions: {len(successful_extractions)}")
print(f"❌ Failed extractions: {len(failed_extractions)}")
print(f"🔄 Documents consolidated: {len(all_metadata) - len(canonical_projects)}")

print(f"\n📋 Final Consolidated Projects:")
for i, (project_name, project_data) in enumerate(canonical_projects.items(), 1):
    doc_count = len(project_data['documents'])
    print(f"  {i}. {project_name}")
    print(f"     Documents: {doc_count}")
    print(f"     Sector: {project_data.get('sector', 'N/A')}")
    if doc_count > 1:
        print(f"     Files: {', '.join([doc['filename'] for doc in project_data['documents']])}")
    print()


INFO:read_dataset:Reading document: 164039.pdf
INFO:read_dataset:Reading document: hodnotenie_d1-bidovce-statna-hranica-sk_ua.pdf


🚀 Starting REAL-TIME document processing with project consolidation...
Using GPT-4.1 for analysis + immediate project matching
Processing documents one by one with live consolidation


INFO:read_dataset:Reading document: Inform cia o zmluvnej cene Ba-Tri.pdf
INFO:read_dataset:Reading document: C_1_10_2_Prinosy_stavby_final.pdf
INFO:read_dataset:Reading document: C_1_4_2_D_Kapacitne_posudenie.pdf
INFO:read_dataset:Reading document: E_1_1a_Stage_Summary_in_English_Language.pdf
INFO:read_dataset:Reading document: Hodnotenie_Najom_MZSR_20250702.pdf
INFO:read_dataset:Reading document: R1-BB-Slovenska-Lupca_Aktualizacia_20230302.pdf
INFO:read_dataset:Reading document: Príloha č. 3 - Podrobné informácie o analyzovaných administratívnych priestoroch.docx.pdf
INFO:read_dataset:Reading document: 2.3_Ortofotomapa_km_0,000-11,500_II._usek.pdf
INFO:read_dataset:Reading document: C_1_5_9_VYSLEDKY_LABORATORNYCH_SKUSOK_ZEMIN_A_HORNIN.pdf
INFO:read_dataset:Reading document: C_1_11_Hodnotenie_Natura_2000.pdf
INFO:read_dataset:Reading document: C_1_1_1_Zivotne_prostredie.pdf
INFO:read_dataset:Reading document: Súčasť prílohy č. 4 - Pôdorys jedáleň + suterén budovy na uli

Processing 103 documents with real-time consolidation...

📄 Processing Document 1/103: 164039.pdf
Content length: 11972 characters


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  📋 Extracted Project: CBA Standard Table - Roads
  🔍 Checking for project matches...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


    🤖 LLM Analysis:
      - Match: False
      - Confidence: high
      - Reasoning: The new project, 'CBA Standard Table - Roads', is a methodological or reference document providing standardized parameters for cost-benefit analysis of road infrastructure projects in Slovakia. It is not a specific infrastructure project, feasibility study, or value-for-money assessment of a particular road, motorway, or transport corridor. None of the existing projects are methodological standards or general reference tables; they are all specific to particular infrastructure projects, locations, or institutions. The only somewhat similar title is 'Standard Table of Vehicles - Long-distance Transport', but that appears to be a standard table for vehicles, not for roads or CBA methodology. Therefore, there is no match.
    🆕 Created new project: CBA Standard Table - Roads
    📊 Decision: No match found - creating new project
    📈 Current Status: 25 consolidated projects
-------------------------------

KeyboardInterrupt: 

In [3]:
# Save consolidated results
print("💾 Saving consolidated project results...")

try:
    # Create a flattened list for Excel export
    consolidated_metadata = []
    
    for project_name, project_data in canonical_projects.items():
        for doc in project_data['documents']:
            consolidated_metadata.append({
                'canonical_project_name': project_name,
                'canonical_project_name_slovak': project_data.get('project_name_slovak', ''),
                'canonical_sector': project_data.get('sector', ''),
                'filename': doc['filename'],
                'original_project_name': doc['project_name'],
                'original_project_name_slovak': doc.get('project_name_slovak', ''),
                'document_type': doc.get('document_type', ''),
                'sector': doc.get('sector', ''),
                'summary': doc.get('summary', ''),
                'content_length': doc.get('content_length', 0),
                'documents_in_project': len(project_data['documents'])
            })
    
    # Save consolidated results
    save_to_excel(consolidated_metadata, "consolidated_projects.xlsx", "Consolidated Projects")
    
    # Save original metadata for comparison
    save_to_excel(all_metadata, "original_metadata.xlsx", "Original Metadata")
    
    # Save to JSON as backup
    with open("consolidated_projects.json", 'w', encoding='utf-8') as f:
        json.dump(canonical_projects, f, ensure_ascii=False, indent=2)
    print("✅ Consolidated projects backup saved to consolidated_projects.json")
    
except Exception as e:
    print(f"❌ Error saving consolidated files: {str(e)}")

print("\n🎉 Consolidated processing complete!")
print("📁 Files created:")
print("- consolidated_projects.xlsx (main output with project groupings)")
print("- original_metadata.xlsx (original document metadata)")
print("- consolidated_projects.json (backup)")
print("\n📊 Key Benefits:")
print("- Documents grouped by actual project relationships")
print("- Canonical project names automatically selected")
print("- Real-time consolidation with LLM + fuzzy matching")
print("- Clear visibility into which documents belong together")


💾 Saving consolidated project results...
✅ Data saved to consolidated_projects.xlsx
✅ Data saved to original_metadata.xlsx
✅ Consolidated projects backup saved to consolidated_projects.json

🎉 Consolidated processing complete!
📁 Files created:
- consolidated_projects.xlsx (main output with project groupings)
- original_metadata.xlsx (original document metadata)
- consolidated_projects.json (backup)

📊 Key Benefits:
- Documents grouped by actual project relationships
- Canonical project names automatically selected
- Real-time consolidation with LLM + fuzzy matching
- Clear visibility into which documents belong together


In [4]:
# Flatten canonical_projects into a document-level DataFrame
consolidated_data = []

for project_name, project_data in canonical_projects.items():
    for doc in project_data['documents']:
        consolidated_data.append({
            'canonical_project_name': project_name,
            'canonical_project_name_slovak': project_data.get('project_name_slovak', ''),
            'filename': doc['filename'],
            'original_project_name': doc['project_name'],
            'original_project_name_slovak': doc.get('project_name_slovak', ''),
            'document_type': doc.get('document_type', ''),
            'sector': doc.get('sector', ''),
            'summary': doc.get('summary', ''),
            'content_length': doc.get('content_length', 0),
            'documents_in_project': len(project_data['documents'])
        })

# Create DataFrame
df_consolidated = pd.DataFrame(consolidated_data)
print(f"DataFrame shape: {df_consolidated.shape}")
df_consolidated.head()

DataFrame shape: (68, 10)


,canonical_project_name,canonical_project_name_slovak,filename,original_project_name,original_project_name_slovak,document_type,sector,summary,content_length,documents_in_project
0,Reconstruction of the Hviezda Dormitory – Stag...,Rekonštrukcia internátu Hviezda – Etapa II.,164039.pdf,Reconstruction of the Hviezda Dormitory – Stag...,Rekonštrukcia internátu Hviezda – Etapa II.,Feasibility Study Update Form (prior to contra...,Building,This document is an update form for the feasib...,11972,2
1,Reconstruction of the Hviezda Dormitory – Stag...,Rekonštrukcia internátu Hviezda – Etapa II.,hodnotenie_hviezda_aktualizacia_pred_podpisom.pdf,Reconstruction of the Hviezda Dormitory – Upda...,Aktualizácia pred podpisom zmluvy: Rekonštrukc...,Update of economic evaluation (cost-benefit an...,Building,This document is an updated economic evaluatio...,4669,2
2,Value for Money Assessment of the D1 Corridor ...,Hodnota za peniaze projektu Koridor D1 Bidovce...,hodnotenie_d1-bidovce-statna-hranica-sk_ua.pdf,Value for Money Assessment of the D1 Corridor ...,Hodnota za peniaze projektu Koridor D1 Bidovce...,Government evaluation of a feasibility study,Transport,This document is an evaluation by the Slovak M...,29963,1
3,Extension of the D1 Motorway Bratislava - Trib...,Rozšírenie diaľnice D1 Bratislava -Triblavina,Inform cia o zmluvnej cene Ba-Tri.pdf,Extension of the D1 Motorway Bratislava - Trib...,Rozšírenie diaľnice D1 Bratislava -Triblavina,Contract price information,Transport,This document provides information about the c...,685,2
4,Extension of the D1 Motorway Bratislava - Trib...,Rozšírenie diaľnice D1 Bratislava -Triblavina,Zhrnutie D1 Bratislava-Triblavina.docx,D1 Motorway Bratislava-Triblavina Expansion,Diaľnica D1 Bratislava-Triblavina,Project summary and technical overview,Transport,This document provides a summary of the planne...,21734,2
